In [12]:
import ipywidgets.widgets as widgets
controller = widgets.Controller(index=0)  #Replace with the index number of the controller you just tested
display(controller)

Controller()

In [13]:
#Import function library path
import sys
sys.path.append('../')
import ipywidgets.widgets as widgets
from servoserial import ServoSerial
from jetbot import Robot
import traitlets
from jetbot import Camera
from jetbot import bgr8_to_jpeg
from jetbot import Heartbeat
import threading
import time
# Thread function operation library
import inspect
import ctypes

import traitlets
from IPython.display import display
from jetbot import Camera, bgr8_to_jpeg
import os
from uuid import uuid1
camera = Camera.instance(width=224, height=224)
image = widgets.Image(format='jpeg', width=224, height=224)  # this width and height doesn't necessarily have to match the camera
camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)
display(image)

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

In [14]:
# 안전성 
robot = Robot()
def handle_heartbeat_status(change):
    if change['new'] == Heartbeat.Status.dead:
        camera_link.unlink() # 카메라 스트림을 안전하게 중단 
        robot.stop()

heartbeat = Heartbeat(period=0.5)
# Append the callback function to the heartbeat state
heartbeat.observe(handle_heartbeat_status, names='status')

In [15]:
def _async_raise(tid, exctype):
    """raises the exception, performs cleanup if needed"""
    tid = ctypes.c_long(tid)
    if not inspect.isclass(exctype):
        exctype = type(exctype)
    res = ctypes.pythonapi.PyThreadState_SetAsyncExc(tid, ctypes.py_object(exctype))
    if res == 0:
        raise ValueError("invalid thread id")
    elif res != 1:
        # """if it returns a number greater than one, you're in trouble,
        # and you should call it again with exc=NULL to revert the effect"""
        ctypes.pythonapi.PyThreadState_SetAsyncExc(tid, None)
        
def stop_thread(thread):
    _async_raise(thread.ident, SystemExit)

In [16]:
'''Servo Part'''
servo_device = ServoSerial() 
def camUpFunction():
    global updownpulse
    updownpulse+=15
    if updownpulse>4095:
        updownpulse=4095
    servo_device.Servo_serial_control(2, updownpulse)

def camDownFunction():
    global updownpulse
    updownpulse-=15
    if updownpulse<2000:
        updownpulse=2000
    servo_device.Servo_serial_control(2, updownpulse)

def camLeftFunction():
    global leftrightpulse
    leftrightpulse+=15
    if leftrightpulse>3600:
        leftrightpulse=3600
    servo_device.Servo_serial_control(1, leftrightpulse)

def camRightFunction():
    global leftrightpulse
    leftrightpulse-=15
    if leftrightpulse<600:
        leftrightpulse=600
    servo_device.Servo_serial_control(1, leftrightpulse)

def camservoInitFunction():
    global leftrightpulse, updownpulse
    leftrightpulse = 2048
    updownpulse = 2048
    servo_device.Servo_serial_control(1, 2048)
    time.sleep(0.1)
    servo_device.Servo_serial_control(2, 2048)

serial Open!


In [17]:
servo_device.Servo_serial_control(2, 4000)

16
b'\xff\xff\x02\x07\x03*\x0f\xa0\x00\n\x10'


In [18]:
servo_device = ServoSerial() 
servo_device.Servo_serial_control(1, 1048)
servo_device.Servo_serial_control(2, 1048)

serial Open!
serial Close!
164
b'\xff\xff\x01\x07\x03*\x04\x18\x00\n\xa4'
166
b'\xff\xff\x02\x07\x03*\x05\x14\x00\n\xa6'


In [20]:
camservoInitFunction()

184
b'\xff\xff\x01\x07\x03*\x08\x00\x00\n\xb8'
183
b'\xff\xff\x02\x07\x03*\x08\x00\x00\n\xb7'


In [21]:
robot = Robot()

In [24]:
import sys
sys.path.append('../')
from RGB_Lib import Programing_RGB
RGB = Programing_RGB()

def BLN_Onboard():
    global i , k
    i = k = 0
    while True:
        if k == 0:
            RGB.Set_ChameleonLight_RGB() # 카멜레온 모드 
        elif k == 1:
            RGB.Set_BreathSColor_RGB(3)
            RGB.Set_BreathSSpeed_RGB(3)
            RGB.Set_BreathSLight_RGB()

In [25]:
thread1 = threading.Thread(target=BLN_Onboard)
# thread1.setDaemon(True)
thread1.start()

In [26]:
def jetbot_motion():
    count1 = count2 = count3 = count4 =  count5 = 0
    while 1:
        #Robot car Left and right DC motor
        #Handle control code---1(Jetbot Yahboom handle)
        #Yahboom Rocker reset value is 0.0039,
        #In order to prevent the motor from being abnormal, the following code is added to operate.
        if controller.axes[1].value <= 0.1:
            if (controller.axes[0].value <= 0.1 and controller.axes[0].value >= -0.1 
                and controller.axes[1].value <= 0.1 and controller.axes[1].value >= -0.1):
                robot.stop()
            else:
                robot.set_motors(-controller.axes[1].value + controller.axes[0].value, -controller.axes[1].value - controller.axes[0].value)
            
            time.sleep(0.01)
        else:
            robot.set_motors(-controller.axes[1].value - controller.axes[0].value, -controller.axes[1].value + controller.axes[0].value)
            time.sleep(0.01)
          #Handle control code---2(Xbox360手柄)
        # if controller.axes[1].value <= 0:
        #     robot.set_motors(-controller.axes[1].value + controller.axes[0].value, -controller.axes[1].value - controller.axes[0].value)
        #     time.sleep(0.01)
        # else:
        #     robot.set_motors(-controller.axes[1].value - controller.axes[0].value, -controller.axes[1].value + controller.axes[0].value)
        #     time.sleep(0.01)

        #Servo control camera up and down 
        if controller.buttons[2].value == True:
            count1  += 1
            if count1 >= 2:
                camDownFunction()
                count1 = 0
        elif controller.buttons[0].value == True:
            count1  += 1
            if count1 >= 2:
                camUpFunction()
                count1 = 0
        else:
            count1 = 0
        #Servo control camera left and right
        if controller.buttons[1].value == True:
            count2  += 1
            if count2 >= 2:
                camRightFunction()
                count2 = 0
        elif controller.buttons[3].value == True:
            count2  += 1
            if count2 >= 2:
                camLeftFunction()
                count2 = 0
        else:
            count2 = 0
        #Servo control camera up down, left and right is reset
        if controller.buttons[8].value == True:
            count3 += 1
            if count3 >= 2:
                camservoInitFunction()
                count3 = 0
        else:
            count3 = 0
        
        #Servo control servo rise and decline
        if controller.buttons[6].value == True:
            count4 += 1
            if count4 >= 3:
                robot.set_vertical_motors(1)
                count4 = 0
        elif controller.buttons[4].value == True:
            count4 += 1
            if count4 >= 3:
                robot.set_vertical_motors(-1)
                count4 = 0
        else:
            robot.set_vertical_motors(0)
            count4 = 0

In [27]:
thread2 = threading.Thread(target=jetbot_motion)
thread2.setDaemon(True)
thread2.start()

In [28]:
stop_thread(thread1)



Set_ChameleonLight_RGB I2C error


In [29]:
stop_thread(thread2)
robot.stop()

In [31]:
camera_link.unlink()

ValueError: list.remove(x): x not in list